<a href="https://colab.research.google.com/github/minhdinh365/DoAn-QLDACNPM/blob/main/Colaboratory_ch%C3%A0o_m%E1%BB%ABng_b%E1%BA%A1n!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from os import listdir
import cv2
import numpy as np
import pickle

import tensorflow
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report



In [6]:
raw_folder = "/content/drive/MyDrive/train"
def save_data(raw_folder=raw_folder):

    dest_size = (128, 128)
    print("Bắt đầu xử lý ảnh...")

    pixels = []
    labels = []

    # Lặp qua các folder con trong thư mục raw
    for folder in listdir(raw_folder):
        index = 0
        if folder!='.DS_Store':
            if index==69:
                break
            print("Folder=",folder)
            # Lặp qua các file trong từng thư mục chứa các em
            for file in listdir(raw_folder  + "/" + folder):
                if file!='.DS_Store':
                    print("File=", file)
                    pixels.append( cv2.resize(cv2.imread(raw_folder + "/" + folder +"/" + file),dsize=(128,128)))
                    labels.append( folder)

    pixels = np.array(pixels)
    labels = np.array(labels)#.reshape(-1,1)

    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    labels = encoder.fit_transform(labels)
    print(labels)

    file = open('pix.data', 'wb')
    # dump information to that file
    pickle.dump((pixels,labels), file)
    # close the file
    file.close()

    return

def load_data():
    file = open('pix.data', 'rb')

    # dump information to that file
    (pixels, labels) = pickle.load(file)

    # close the file
    file.close()

    print(pixels.shape)
    print(labels.shape)


    return pixels, labels

#save_data()
X,y = load_data()

#random.shuffle(X)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15)



def get_model():
    model_vgg16_conv = VGG19(weights='imagenet', include_top=False)

    # Dong bang cac layer
    for layer in model_vgg16_conv.layers:
        layer.trainable = False

    # Tao model
    input = Input(shape=(128, 128, 3), name='image_input')
    output_vgg16_conv = model_vgg16_conv(input)
    # Them cac layer FC va Dropout
    x = Flatten(name='flatten')(output_vgg16_conv)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dropout(0.25)(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dropout(0.25)(x)
    x = Dense(10, activation='softmax', name='predictions')(x)

    # Compile
    my_model = Model(inputs=input, outputs=x)
    my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return my_model


vggmodel = get_model()

filepath="weights-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.1,
    rescale=1./255,
	width_shift_range=0.1,
    height_shift_range=0.1,
	horizontal_flip=True,
    brightness_range=[0.2,1.5], fill_mode="nearest")

aug_val = ImageDataGenerator(rescale=1./255)

# print(X_train.shape)
# print(y_train.shape)
# print(X_test.shape)
# print(y_test.shape)
# print(callbacks_list)
# X_train = X_train.reshape(-1, 219, 786432, 1)
# y_train = y_train.reshape(-1, 219, 4, 1)
# X_test = X_test.reshape(-1, 55, 786432, 1)
# y_test = y_test.reshape(-1, 55, 4, 1)
# X_train = tensorflow.keras.utils.to_categorical(X_train, num_classes= 4)
# y_train = tensorflow.keras.utils.to_categorical(y_train, 4)
# X_test = tensorflow.keras.utils.to_categorical(X_test, 4)
# y_test = tensorflow.keras.utils.to_categorical(y_test, 4)
vgghist=vggmodel.fit_generator(aug.flow(X_train, y_train, batch_size=64),
                               epochs=100,# steps_per_epoch=len(X_train)//64,
                               validation_data=aug.flow(X_test,y_test,
                               batch_size=64),
                               callbacks=callbacks_list)
vggmodel.save("vggmodel.h5")
vgghist.save("vggmodelnew.h5")

(9000, 128, 128, 3)
(9000, 10)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
113/113 [==============================] - 41s 360ms/step - loss: 4.6404 - accuracy: 0.2184 - val_loss: 1.4759 - val_accuracy: 0.5050

Epoch 00001: val_accuracy improved from -inf to 0.50500, saving model to weights-01-0.50.hdf5
Epoch 2/100
113/113 [==============================] - 39s 349ms/step - loss: 1.4918 - accuracy: 0.4879 - val_loss: 1.3463 - val_accuracy: 0.5339

Epoch 00002: val_accuracy improved from 0.50500 to 0.53389, saving model to weights-02-0.53.hdf5
Epoch 3/100
113/113 [==============================] - 39s 348ms/step - loss: 1.3808 - accuracy: 0.5147 - val_loss: 1.3705 - val_accuracy: 0.5283

Epoch 00003: val_accuracy did not improve from 0.53389
Epoch 4/100
113/113 [==============================] - 39s 349ms/step - loss: 1.3646 - accuracy: 0.5297 - val_loss: 1.2900 - val_accuracy: 0.5511

Epoch 00004: val_accuracy improved from 0.53389 to 0.55111, saving model to weights-04-0.55.hdf5
Epoch 5/100
113/113 [==============================] - 40s 350ms/step

AttributeError: ignored

In [9]:
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Flatten, Dense, Dropout
import keras
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import random
import tensorflow as tf

In [35]:
class_names = ['chef','doctor','engineer','farmer','firefighter','judge','mechanic','pilot','police','waiter']

model=tf.keras.models.load_model('weights-56-0.65.hdf5')
imgpath='/content/drive/MyDrive/test/doctor/doctor-103.jpg'
img = keras.preprocessing.image.load_img(
    imgpath, target_size=(128, 128)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict( img_array)
print(predictions)
score = tf.nn.softmax(predictions)
print((score))
print(
    "Hình ảnh {} với độ chính xác {:.2f} phần trăm."
    .format(class_names[np.argmax(score)], 100 * np.max(predictions))
)

[[0.         0.9825676  0.         0.         0.         0.
  0.01743242 0.         0.         0.        ]]
tf.Tensor(
[[0.08555131 0.22853377 0.08555131 0.08555131 0.08555131 0.08555131
  0.08705575 0.08555131 0.08555131 0.08555131]], shape=(1, 10), dtype=float32)
Hình ảnh doctor với độ chính xác 98.26 phần trăm.


In [ ]:
def plot_model_history(model_history, acc='accuracy', val_acc='val_accuracy'):
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    axs[0].plot(range(1, len(model_history.history[acc]) + 1), model_history.history[acc])
    axs[0].plot(range(1, len(model_history.history[val_acc]) + 1), model_history.history[val_acc])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1, len(model_history.history[acc]) + 1), len(model_history.history[acc]) / 10)
    axs[0].legend(['train', 'val'], loc='best')
    axs[1].plot(range(1, len(model_history.history['loss']) + 1), model_history.history['loss'])
    axs[1].plot(range(1, len(model_history.history['val_loss']) + 1), model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1, len(model_history.history['loss']) + 1), len(model_history.history['loss']) / 10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()
    plt.savefig('roc.png')